In [134]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [135]:
import glob
import pickle
from unittest import result
import numpy as np
import random
from tqdm import tqdm
from typing import cast
from music21 import converter, instrument, note, chord, stream
from music21.stream.base import Score
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization, Input, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [136]:
def prepare_sequences(notes, total_comp, n_vocab):
	sequence_length = 100

	# create a dictionary to map vocab to integers
	note_index_map = dict((note, number) for number, note in enumerate(total_comp))

	network_input = []
	network_output = []

	# create input sequences and outputs
	for i in range(0, len(notes) - sequence_length, 1):
		sequence_input = notes[i:i + sequence_length]
		sequence_output = notes[i + sequence_length]
		network_input.append([note_index_map[char] for char in sequence_input])
		network_output.append(note_index_map[sequence_output])

	n_patterns = len(network_input)

	# reshape for LSTM compatibility
	normalised_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

	# normalise input
	normalised_input = normalised_input / float(n_vocab)


	return (network_input, normalised_input)

In [137]:
def create_input_branch(input_data, name, lstm_units=256, dropout_rate=0.2):

    input_layer = Input(shape=(input_data.shape[1], input_data.shape[2]), name=f"input_{name}")
    x = LSTM(
        lstm_units,
        return_sequences=True,
        name=f"lstm_{name}"
    )(input_layer)
    x = Dropout(dropout_rate, name=f"dropout_{name}")(x)

    return input_layer, x

In [138]:
def create_output_branch(x, n_vocab, name, dense_units=128, dropout_rate=0.3):

    x = Dense(dense_units, activation='relu', name=f"dense_{name}")(x)
    x = BatchNormalization(name=f"bn_{name}")(x)
    x = Dropout(dropout_rate, name=f"dropout_{name}")(x)
    output = Dense(n_vocab, activation='softmax', name=name)(x)

    return output

In [139]:
def create_network(network_input_notes, n_vocab_notes,
                  network_input_durations, n_vocab_durations,
                  network_input_offsets, n_vocab_offsets):

    # Create input branches
    input_notes_layer, input_notes = create_input_branch(network_input_notes, "notes")
    input_durations_layer, input_durations = create_input_branch(network_input_durations, "durations")
    input_offsets_layer, input_offsets = create_input_branch(network_input_offsets, "offsets")

    # Concatenate the three input branches
    combined = concatenate([input_notes, input_durations, input_offsets], name="combined_features")

    # Process combined features
    x = LSTM(512, return_sequences=True, name="lstm_combined_1")(combined)
    x = Dropout(0.3, name="dropout_combined_1")(x)
    x = LSTM(512, name="lstm_combined_2")(x)
    x = BatchNormalization(name="bn_combined")(x)
    x = Dropout(0.3, name="dropout_combined_2")(x)
    x = Dense(256, activation='relu', name="dense_combined")(x)

    # Create output branches
    output_notes = create_output_branch(x, n_vocab_notes, "Note")
    output_durations = create_output_branch(x, n_vocab_durations, "Duration")
    output_offsets = create_output_branch(x, n_vocab_offsets, "Offset")

    # Create and compile model
    model = Model(
        inputs=[input_notes_layer, input_durations_layer, input_offsets_layer],
        outputs=[output_notes, output_durations, output_offsets]
    )

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.load_weights('/content/drive/MyDrive/finalyearproject/models/other.keras')
    # model.load_weights('/content/drive/MyDrive/finalyearproject/models/finalmodel.keras')

    return model

In [140]:
def convert_to_float(value_str):
    try:
        if '/' in value_str:
            num, denom = value_str.split('/')
            return float(num) / float(denom)
        else:
            return float(value_str)
    except ValueError:
        # Default value if parsing fails
        return 0.5

In [141]:
def generate_notes(model, network_input_notes, network_input_durations, network_input_offsets,
                   pitchnames, durationames, offsetnames, n_vocab_notes, n_vocab_durations, n_vocab_offsets,
                   num_predictions=150):

    int_note = dict(enumerate(pitchnames))
    int_duration = dict(enumerate(durationames))
    int_offset = dict(enumerate(offsetnames))

    # Pick random starting sequences
    start_indices = {
        'notes': np.random.randint(0, len(network_input_notes) - 1),
        'durations': np.random.randint(0, len(network_input_durations) - 1),
        'offsets': np.random.randint(0, len(network_input_offsets) - 1)
    }

    # Initialize patterns
    patterns = {
        'notes': network_input_notes[start_indices['notes']].copy(),
        'durations': network_input_durations[start_indices['durations']].copy(),
        'offsets': network_input_offsets[start_indices['offsets']].copy()
    }

    prediction_output = []

    #Generate notes
    for _ in range(num_predictions):
        # Prepare inputs for prediction
        inputs = [
            np.reshape(patterns['notes'], (1, len(patterns['notes']), 1)) / float(n_vocab_notes),
            np.reshape(patterns['durations'], (1, len(patterns['durations']), 1)) / float(n_vocab_durations),
            np.reshape(patterns['offsets'], (1, len(patterns['offsets']), 1)) / float(n_vocab_offsets)
        ]

        # Get the current note
        current_note = int_note[patterns['notes'][-1]]

        # Get model predictions
        predictions = model.predict(inputs, verbose=0)

        # Extract predicted indices
        indices = {
            'note': np.argmax(predictions[0]),
            'duration': np.argmax(predictions[1]),
            'offset': np.argmax(predictions[2])
        }

        #convert indices to values
        results = {
            'note': int_note[indices['note']],
            'duration': int_duration[indices['duration']],
            'offset': int_offset[indices['offset']]
        }

        # prediction info
        print(f"Next note: {current_note} - Duration: {results['duration']} - Offset: {results['offset']}")

        # Add to output
        prediction_output.append([results['note'], results['duration'], results['offset']])

        # Update patterns for next iteration (shift window)
        for key, pattern in patterns.items():
            pattern.append(indices[key.rstrip('s')])  # remove trailing 's' to match indices keys
            patterns[key] = pattern[1:]

    return prediction_output


In [142]:
def create_midi(prediction_output_all):
    output_notes = []
    current_offset = 0

    #extract the components from prediction output
    notes = []
    durations = []
    offsets = []

    print("Processing prediction output...")
    for item in prediction_output_all:
        print(item)
        notes.append(item[0])

        # Process durations - handle both numeric and fraction formats
        try:
            durations.append(float(item[1]))
        except ValueError:
            if '/' in item[1]:
                num, denom = item[1].split('/')
                durations.append(float(num) / float(denom))
            else:
                durations.append(1.0)  # Default value

        # Process offsets
        try:
            offsets.append(float(item[2]))
        except ValueError:
            if '/' in item[2]:
                num, denom = item[2].split('/')
                offsets.append(float(num) / float(denom))
            else:
                offsets.append(0.5)  # default value

    print("---")
    print(f"Notes: {notes}")
    print(f"Offsets: {offsets}")
    print(f"Durations: {durations}")
    print("generating Midi File...")

    # Create note and chord objects based on the values generated by the model
    for i, pattern in enumerate(notes):
        #  duration - handle both numeric and fraction formats
        try:
            duration_value = float(durations[i])
        except (ValueError, TypeError):
            if isinstance(durations[i], str) and '/' in durations[i]:
                num, denom = durations[i].split('/')
                duration_value = float(num) / float(denom)
            else:
                duration_value = 1.0  # Default value

        # Create either a chord or note
        if ('.' in pattern) or pattern.isdigit():
            # Handle chord
            notes_in_chord = pattern.split('.')
            chord_notes = []

            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                chord_notes.append(new_note)

            new_chord = chord.Chord(chord_notes)
            new_chord.duration.quarterLength = duration_value
            new_chord.offset = current_offset
            output_notes.append(new_chord)
        else:
            # Handle single note
            new_note = note.Note(pattern)
            new_note.storedInstrument = instrument.Piano()
            new_note.duration.quarterLength = duration_value
            new_note.offset = current_offset
            output_notes.append(new_note)

        #update offset for next note
        try:
            current_offset += float(offsets[i])
        except (ValueError, TypeError):
            if isinstance(offsets[i], str) and '/' in offsets[i]:
                num, denom = offsets[i].split('/')
                current_offset += float(num) / float(denom)
            else:
                current_offset += 0.5  # Default value

    # Create MIDI stream and write to file
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=Path("/content/drive/MyDrive/finalyearproject/x.mid"))

    print("Midi created!")

In [143]:
def generate():
    notes_file = Path("/content/drive/MyDrive/finalyearproject/notes/notes7.pkl")
    # notes_file = Path("/content/drive/MyDrive/finalyearproject/notes/notes6.pkl")
    notes_combined = []
    with open(notes_file, "rb") as file:
        notes_combined = pickle.load(file)

    # Extract individual components and convert to appropriate types
    all_pitches = []
    all_durations = []
    all_offsets = []
    for item in notes_combined:
        parts = item.split(":")
        all_pitches.append(parts[0])

        # Convert duration string to float
        duration_str = parts[1]
        if '/' in duration_str:
            num, denom = duration_str.split('/')
            duration = float(num) / float(denom)
        else:
            duration = float(duration_str)
        all_durations.append(duration)

        # Convert offset string to float
        offset_str = parts[2]
        if '/' in offset_str:
            num, denom = offset_str.split('/')
            offset = float(num) / float(denom)
        else:
            offset = float(offset_str)
        all_offsets.append(offset)

    pitchnames = sorted(set(all_pitches))
    durations = sorted(set(all_durations))
    offsets = sorted(set(all_offsets))

    n_vocab_offsets = len(offsets)
    network_input_offsets, normalised_input_offsets = prepare_sequences(all_offsets, offsets, n_vocab_offsets)

    n_vocab_notes = len(pitchnames)
    network_input_notes, normalised_input_notes = prepare_sequences(all_pitches, pitchnames, n_vocab_notes)

    n_vocab_durations = len(durations)
    network_input_durations, normalised_input_durations = prepare_sequences(all_durations, durations, n_vocab_durations)

    model = create_network(normalised_input_notes, n_vocab_notes, normalised_input_durations, n_vocab_durations, normalised_input_offsets, n_vocab_offsets)
    # model = create_network()
    prediction_output = generate_notes(model, network_input_notes, network_input_durations, network_input_offsets, pitchnames, durations, offsets, n_vocab_notes, n_vocab_durations, n_vocab_offsets)
    create_midi(prediction_output)

In [145]:
generate()

Next note: C4 - Duration: 0.25 - Offset: 0.25
Next note: F3 - Duration: 0.25 - Offset: 0.25
Next note: F3 - Duration: 0.5 - Offset: 0.0
Next note: 5.8.0 - Duration: 0.25 - Offset: 0.0
Next note: G3 - Duration: 0.5 - Offset: 0.25
Next note: F3 - Duration: 0.25 - Offset: 0.5
Next note: C3 - Duration: 0.25 - Offset: 0.25
Next note: 5.8 - Duration: 0.25 - Offset: 0.5
Next note: F3 - Duration: 1.25 - Offset: 0.0
Next note: C3 - Duration: 0.25 - Offset: 0.5
Next note: C3 - Duration: 0.25 - Offset: 0.0
Next note: F3 - Duration: 0.25 - Offset: 0.25
Next note: C4 - Duration: 0.25 - Offset: 0.25
Next note: E-3 - Duration: 0.25 - Offset: 0.25
Next note: E-3 - Duration: 0.25 - Offset: 0.25
Next note: E-3 - Duration: 0.25 - Offset: 0.25
Next note: E-3 - Duration: 0.5 - Offset: 0.25
Next note: E-3 - Duration: 0.25 - Offset: 0.5
Next note: 3.5 - Duration: 0.25 - Offset: 0.5
Next note: F3 - Duration: 0.25 - Offset: 0.25
Next note: B-2 - Duration: 0.25 - Offset: 0.25
Next note: 5.7 - Duration: 0.25 - O